In [1]:
#import data
import pandas as pd
data_base = pd.read_csv("changed_selected_93_08_NIGMS_R01_base_1993ppid.csv",encoding="latin-1")

In [2]:
#check column names
data_base.columns

Index(['NIH.Spending.Categorization', 'Project.Abstract', 'Project.Terms',
       'Project.Title', 'Public.Health.Relevance', 'Administering..IC',
       'Application.ID', 'Award.Notice.Date', 'FOA', 'Project.Number', 'Type',
       'Activity', 'IC', 'Serial.Number', 'Support.Year', 'Suffix',
       'Program.Official.Information', 'Project.Start.Date',
       'Project.End.Date', 'Study.Section', 'Subproject.Number',
       'Contact.PI..Person.ID', 'Contact.PI...Project.Leader',
       'Other.PI.or.Project.Leader.s.', 'Congressional.District', 'Department',
       'DUNS.Number', 'FIPS', 'Latitude', 'Longitude', 'Organization.Name',
       'Organization.City', 'Organization.State', 'Organization.Type',
       'Organization.Zip', 'Organization.Country', 'ARRA.Indicator',
       'Budget.Start.Date', 'Budget.End.Date', 'CFDA.Code',
       'Funding.Mechanism', 'FY', 'Funding.IC', 'FY.Direct.Costs',
       'FY.Indirect.Costs', 'FY.Total.Cost.by.IC',
       'FY.Total.Cost..Sub.Projects.'],
   

In [4]:
#select useful columns
database=data_base[['Type','FY','Project.End.Date','Budget.Start.Date','Project.Number','Support.Year']]
database.columns=['Type','FY','Project_End_Date','Budget_Start_Date','Project_Number','Support_Year']
#add a new column for unique project number by identify unique parts in each project numbers
database['Project_Number_New']=data_base['Project.Number'].str[6:12].astype(int)
database.head()

/Users/lijingning/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Type,FY,Project_End_Date,Budget_Start_Date,Project_Number,Support_Year,Project_Number_New
0,5,1993,7/31/96,8/1/93,5R01GM046926-02,2,46926
1,1,1993,12/31/96,1/1/93,1R01GM048560-01,1,48560
2,5,1993,4/30/96,5/1/93,5R01GM047286-02,2,47286
3,5,1993,3/31/95,4/1/93,5R01GM036143-07,7,36143
4,1,1993,7/31/94,8/1/93,1R01GM049321-01,1,49321


In [5]:
#select competing applications and sort data by project number
database=database[(database.Type == 1) | (database.Type == 2) | (database.Type == 7)|(database.Type == 9)]
database=database.sort(['Project_Number_New','FY'], ascending=True).reset_index(drop=True)
database.head()

/Users/lijingning/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,Type,FY,Project_End_Date,Budget_Start_Date,Project_Number,Support_Year,Project_Number_New
0,2,1996,5/31/00,6/1/96,2R01GM000091-51,51,91
1,2,2000,5/31/04,6/1/00,2R01GM000091-55,55,91
2,2,2004,5/31/08,6/10/04,2R01GM000091-59,59,91
3,2,1993,11/30/96,12/1/92,2R01GM004725-37,37,4725
4,2,1994,3/31/97,4/1/94,2R01GM005147-37A1,37,5147


In [6]:
len(database)

6350

In [7]:
database['Project_End_Date'] = pd.to_datetime(database['Project_End_Date'])
database['Budget_Start_Date'] = pd.to_datetime(database['Budget_Start_Date'])

In [8]:
#convert years into months for later gap length calculation
def months_between(date1,date2):
    m1 = date1.year*12 + date1.month
    m2 = date2.year*12 + date2.month
    months = m2 - m1
    if date1.day > date2.day:
        months = months - 1
    if months < 0: 
        months = 0
    return months

In [9]:
from datetime import datetime,timedelta
s='20081231'
date=datetime(year=int(s[0:4]),month=int(s[4:6]),day=int(s[6:8]))

In [10]:
#calculate gaps by each row
gap = []
for i in range(len(database)-1):
    n = database.Project_Number_New.tolist()
    gsd=database.Project_End_Date.tolist()
    ged=database.Budget_Start_Date.tolist()
    if n[i+1]-n[i]==0:
        a=months_between(gsd[i],ged[i+1])
        gap.append(a) #calculate gap for each project
    else:
        if gsd[i].year <= 2008: #check if the last record for each project is before 2008
            b=months_between(gsd[i],date)
            gap.append(b) 
        else:
            gap.append(0) #if the last record happens after 2008 there's no gap
len(gap)

6349

In [11]:
#check the last record of this data frame
database.tail()

,Type,FY,Project_End_Date,Budget_Start_Date,Project_Number,Support_Year,Project_Number_New
6345,1,2008,2012-08-31,2008-09-10,1R01GM086639-01,1,86639
6346,9,2008,2012-07-31,2008-09-30,9R01GM086854-09A1,9,86854
6347,9,2008,2012-06-30,2008-07-01,9R01GM086877-04A2,4,86877
6348,9,2008,2012-07-31,2008-09-19,9R01GM087137-06,6,87137
6349,1,2008,2012-07-31,2008-09-22,1R01GM087738-01A1,1,87738


In [12]:
#insert the last value of gap manually since the function loop cannot get the last gap value
gap.insert(6349,0)
len(gap)

6350

In [13]:
#add gap column
Gap=pd.Series(gap)
database['Gap'] = Gap.values
database.head()

,Type,FY,Project_End_Date,Budget_Start_Date,Project_Number,Support_Year,Project_Number_New,Gap
0,2,1996,2000-05-31,1996-06-01,2R01GM000091-51,51,91,0
1,2,2000,2004-05-31,2000-06-01,2R01GM000091-55,55,91,0
2,2,2004,2008-05-31,2004-06-10,2R01GM000091-59,59,91,7
3,2,1993,1996-11-30,1992-12-01,2R01GM004725-37,37,4725,145
4,2,1994,1997-03-31,1994-04-01,2R01GM005147-37A1,37,5147,0


In [14]:
#mark re-entry by each row
re_entry=[]
for i in range(len(database)-1):
    n=database.Project_Number_New.tolist()
    gap=database.Gap.tolist()
    gsd=database.Project_End_Date.tolist()
    ged=database.Budget_Start_Date.tolist()
    if n[i+1]-n[i]!=0:
        if gsd[i].year < 2008: #only if the last record of each project is before 2008 will the re_entry be 0
            re_entry.append(0)
        else:
            re_entry.append(1) #projects always come back in other situations: re_entry=1
    else:
        re_entry.append(1)
len(re_entry)

6349

In [15]:
re_entry.insert(6349,1)
len(re_entry)

6350

In [16]:
#add re-entry column
Re_entry=pd.Series(re_entry)
database['Re_Entry'] = Re_entry.values
database.head()

,Type,FY,Project_End_Date,Budget_Start_Date,Project_Number,Support_Year,Project_Number_New,Gap,Re_Entry
0,2,1996,2000-05-31,1996-06-01,2R01GM000091-51,51,91,0,1
1,2,2000,2004-05-31,2000-06-01,2R01GM000091-55,55,91,0,1
2,2,2004,2008-05-31,2004-06-10,2R01GM000091-59,59,91,7,1
3,2,1993,1996-11-30,1992-12-01,2R01GM004725-37,37,4725,145,0
4,2,1994,1997-03-31,1994-04-01,2R01GM005147-37A1,37,5147,0,1


In [17]:
database.to_csv('NIH_93_08_R01_R37_anyFunding_4_1.csv', index = False)

In [18]:
#create new data frame for gaps>=12 months and re-entry
data_new=database[['Gap','Re_Entry']]
data_new=data_new[data_new.Gap >= 12]
data_new.head()

,Gap,Re_Entry
3,145,0
6,81,0
7,12,1
8,94,0
9,102,0


In [19]:
data_new.groupby('Re_Entry').count()

,Gap
Re_Entry,
0,1931
1,567


In [20]:
#calculate total re-entry proportion
total_re_entry=567/(567+1931)
total_re_entry

0.22698158526821458

In [21]:
#count numbers for every gap lengths
Count=data_new.groupby('Gap').count()
Count.reset_index(level=0, inplace=True)
Count

,Gap,Re_Entry
0,12,126
1,13,58
2,14,27
3,15,54
4,16,55
5,17,59
6,18,50
7,19,32
8,20,47
9,21,50


In [22]:
#test re-entry proportion data frame for gap=35
G1=data_new.loc[data_new['Gap'] == 35]
test_df=G1.groupby('Re_Entry').count()
test_df

,Gap
Re_Entry,
0,5
1,4


In [23]:
#a little test for the code getting re_entry count for specific gap length
#this code will be used in the next loop
xx=Count.loc[Count['Gap'] == 12, 'Re_Entry'].iloc[0]
xx

126

In [24]:
#loop for re-entry fracton for each gap length
fraction=[]
gaps=Count.Gap.tolist()
for i in gaps:
    g=data_new.loc[data_new['Gap']==i] #get re_entry records in data_new for each gap length
    p=g.groupby('Re_Entry').count() #p is the proprotion df for each gap length like the test_df before
    n=Count.loc[Count['Gap'] == i, 'Re_Entry'].iloc[0] #n is the count of re_entry(both 1 and 0) for each gap length
    if len(p.index)==2: 
        d=p.iloc[1]['Gap'] #d if the count of re_entry=1
        fraction.append(d/n) #count(re_entry=1)/count(re_entry=1 and 0)
    else:
        r = p.index[0] #if the proportion df has only 1 value for re_entry, identify the value is 1 or 0
        if r==0:
            fraction.append(0)
        else:
            fraction.append(1)
fraction

[0.91269841269841268,
 0.60344827586206895,
 1,
 1,
 0.61818181818181817,
 0.61016949152542377,
 0.34000000000000002,
 0.8125,
 0.46808510638297873,
 0.38,
 0.8125,
 0.5,
 0.67346938775510201,
 0.5,
 1,
 1,
 0.27777777777777779,
 0.25,
 0.20689655172413793,
 0.5,
 0.30769230769230771,
 0.076923076923076927,
 0,
 0.44444444444444442,
 0.31578947368421051,
 0.29411764705882354,
 1,
 1,
 0.16666666666666666,
 0.055555555555555552,
 0.09375,
 0.35714285714285715,
 0.23809523809523808,
 0.073170731707317069,
 0.1111111111111111,
 0,
 0.18181818181818182,
 0,
 1,
 1,
 0.066666666666666666,
 0,
 0.034482758620689655,
 0.1111111111111111,
 0,
 0.024390243902439025,
 0,
 0,
 0.0625,
 0.076923076923076927,
 1,
 0,
 0.045454545454545456,
 0,
 0,
 0,
 0,
 0.066666666666666666,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.16666666666666666,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.26923076923076922,
 0.0

In [25]:
len(fraction)

146

In [26]:
len(gaps)

146

In [29]:
# export the re-rntry fraction to a csv
data_frame = pd.DataFrame()
dt = [gaps, fraction]
data_frame = pd.DataFrame(dt)
data_frame = data_frame.transpose()
data_frame.columns = [ "Gap",'Fraction']
data_frame   

,Gap,Fraction
0,12.0,0.912698
1,13.0,0.603448
2,14.0,1.000000
3,15.0,1.000000
4,16.0,0.618182
5,17.0,0.610169
6,18.0,0.340000
7,19.0,0.812500
8,20.0,0.468085
9,21.0,0.380000


In [30]:
data_frame.to_csv('Re-entry_Fraction.csv', index = False)